<a href="https://colab.research.google.com/github/Aviral-Aggarwal/Retail_models/blob/master/Retail_basic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#New version of numpy is there, using the upgraded one
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

warnings.filterwarnings('ignore')

In [ ]:
input_dir = '/content/gdrive/My Drive/m5-forecasting-accuracy/'

#CHECK AGAIN HOW THIS IS WORKING, IS IT ZIP MECHANISM OR SOMETHING ELSE
#Standard function, copied from stack exchange
def reduce_mem_usage(df, verbose=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
def load_df(file_path):
    t1 = time.time()
    df = pd.read_csv(file_path)
    df = reduce_mem_usage(df=df, verbose=True)
    print('The shape of data frame is ', df.shape)
    t2 = time.time()
    print('\nTime taken to load the DataFrame is:', (t2-t1))
    return df

In [ ]:
#Don't load and run this one, start from loading food_xgb
food = load_df(input_dir + 'household1_master.csv')

Mem. usage decreased to 914.67 Mb (65.1% reduction)
The shape of data frame is  (7797559, 44)

Time taken to load the DataFrame is: 54.98212480545044


I am dropping some of the columns for the following reasons:
1. id: combination of item_id and store_id
2. dept_id: name of the category
3. weekday: wday is already present as a number
4. wm_yr_wk_x, wm_yr_wk_y : a representation of the dates
5. d_num: indirectly just sequencing of the time series data
6. event_name_1: already converted into columns
7. date_y: repeat column

In [ ]:
food.drop(food.columns[0], axis=1, inplace=True)

In [ ]:
food.drop(['id', 'd', 'weekday', 'event_name_1'], axis=1, inplace=True)

KeyError: ignored

In [ ]:
food

,item_id,store_id,id,d,Quantity_sold,wm_yr_wk_x,sell_price,weekday,wday,month,year,mrp,discount,week_number,event_name_1,PresidentsDay,LentStart,LentWeek2,StPatricksDay,Purim End,OrthodoxEaster,Pesach End,Cinco De Mayo,Mother's day,MemorialDay,NBAFinalsStart,NBAFinalsEnd,Father's day,IndependenceDay,Ramadan starts,Eid al-Fitr,LaborDay,ColumbusDay,Halloween,EidAlAdha,VeteransDay,Thanksgiving,Christmas,Chanukah End,NewYear,OrthodoxChristmas,MartinLutherKingDay,Easter
0,HOUSEHOLD_1_001,CA_1,HOUSEHOLD_1_001_CA_1_validation,d_1,3,11101,6.320312,Saturday,1,1,2011,6.320312,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOUSEHOLD_1_001,CA_1,HOUSEHOLD_1_001_CA_1_validation,d_2,0,11101,6.320312,Sunday,2,1,2011,6.320312,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOUSEHOLD_1_001,CA_1,HOUSEHOLD_1_001_CA_1_validation,d_3,0,11101,6.320312,Monday,3,1,2011,6.320312,0.0,5,NaN,-21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOUSEHOLD_1_001,CA_1,HOUSEHOLD_1_001_CA_1_validation,d_4,0,11101,6.320312,Tuesday,4,2,2011,6.320312,0.0,5,NaN,-20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOUSEHOLD_1_001,CA_1,HOUSEHOLD_1_001_CA_1_validation,d_5,0,11101,6.320312,Wednesday,5,2,2011,6.320312,0.0,5,NaN,-19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7797554,HOUSEHOLD_1_541,WI_3,HOUSEHOLD_1_541_WI_3_validation,d_1909,0,11612,6.968750,Wednesday,5,4,2016,7.968750,1.0,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7797555,HOUSEHOLD_1_541,WI_3,HOUSEHOLD_1_541_WI_3_validation,d_1910,0,11612,6.968750,Thursday,6,4,2016,7.968750,1.0,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7797556,HOUSEHOLD_1_541,WI_3,HOUSEHOLD_1_541_WI_3_validation,d_1911,0,11612,6.968750,Friday,7,4,2016,7.968750,1.0,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7797557,HOUSEHOLD_1_541,WI_3,HOUSEHOLD_1_541_WI_3_validation,d_1912,0,11613,6.968750,Saturday,1,4,2016,7.968750,1.0,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
food_test = food[(food['wm_yr_wk_x']>11600)]
food_train = food[(food['wm_yr_wk_x']<11600)]

In [ ]:
food_test.drop('wm_yr_wk_x', axis=1, inplace=True)

In [ ]:
food_train.drop('wm_yr_wk_x', axis=1, inplace=True)

In [ ]:
food_train

,item_id,store_id,Quantity_sold,sell_price,wday,month,year,mrp,discount,week_number,PresidentsDay,LentStart,LentWeek2,StPatricksDay,Purim End,OrthodoxEaster,Pesach End,Cinco De Mayo,Mother's day,MemorialDay,NBAFinalsStart,NBAFinalsEnd,Father's day,IndependenceDay,Ramadan starts,Eid al-Fitr,LaborDay,ColumbusDay,Halloween,EidAlAdha,VeteransDay,Thanksgiving,Christmas,Chanukah End,NewYear,OrthodoxChristmas,MartinLutherKingDay,Easter
0,HOUSEHOLD_1_001,CA_1,3,6.320312,1,1,2011,6.320312,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOUSEHOLD_1_001,CA_1,0,6.320312,2,1,2011,6.320312,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOUSEHOLD_1_001,CA_1,0,6.320312,3,1,2011,6.320312,0.0,5,-21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOUSEHOLD_1_001,CA_1,0,6.320312,4,2,2011,6.320312,0.0,5,-20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOUSEHOLD_1_001,CA_1,0,6.320312,5,2,2011,6.320312,0.0,5,-19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7797468,HOUSEHOLD_1_541,WI_3,1,6.968750,3,1,2016,7.968750,1.0,4,-21.0,-16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN
7797469,HOUSEHOLD_1_541,WI_3,0,6.968750,4,1,2016,7.968750,1.0,4,-20.0,-15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN
7797470,HOUSEHOLD_1_541,WI_3,2,6.968750,5,1,2016,7.968750,1.0,4,-19.0,-14.0,-21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN
7797471,HOUSEHOLD_1_541,WI_3,0,6.968750,6,1,2016,7.968750,1.0,4,-18.0,-13.0,-20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN


There are supposed to be 38 columns at this point of time

In [ ]:
food_train.to_csv(input_dir + 'household1_train.csv')
food_test.to_csv(input_dir + 'household1_test.csv')

In [ ]:
train = load_df(input_dir + 'household1_train.csv')
test = load_df(input_dir + 'household1_test.csv')

Mem. usage decreased to 623.01 Mb (71.5% reduction)
The shape of data frame is  (7340123, 39)

Time taken to load the DataFrame is: 36.03771495819092
Mem. usage decreased to 93.36 Mb (31.4% reduction)
The shape of data frame is  (457436, 39)

Time taken to load the DataFrame is: 1.7898826599121094


In [ ]:
7340213*0.667

4895922.071

In [ ]:
train

,Unnamed: 0,item_id,store_id,Quantity_sold,sell_price,wday,month,year,mrp,discount,week_number,PresidentsDay,LentStart,LentWeek2,StPatricksDay,Purim End,OrthodoxEaster,Pesach End,Cinco De Mayo,Mother's day,MemorialDay,NBAFinalsStart,NBAFinalsEnd,Father's day,IndependenceDay,Ramadan starts,Eid al-Fitr,LaborDay,ColumbusDay,Halloween,EidAlAdha,VeteransDay,Thanksgiving,Christmas,Chanukah End,NewYear,OrthodoxChristmas,MartinLutherKingDay,Easter
0,0,HOUSEHOLD_1_001,CA_1,3,6.320312,1,1,2011,6.320312,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,HOUSEHOLD_1_001,CA_1,0,6.320312,2,1,2011,6.320312,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,HOUSEHOLD_1_001,CA_1,0,6.320312,3,1,2011,6.320312,0.0,5,-21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,HOUSEHOLD_1_001,CA_1,0,6.320312,4,2,2011,6.320312,0.0,5,-20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,HOUSEHOLD_1_001,CA_1,0,6.320312,5,2,2011,6.320312,0.0,5,-19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7340118,7797468,HOUSEHOLD_1_541,WI_3,1,6.968750,3,1,2016,7.968750,1.0,4,-21.0,-16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN
7340119,7797469,HOUSEHOLD_1_541,WI_3,0,6.968750,4,1,2016,7.968750,1.0,4,-20.0,-15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN
7340120,7797470,HOUSEHOLD_1_541,WI_3,2,6.968750,5,1,2016,7.968750,1.0,4,-19.0,-14.0,-21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN
7340121,7797471,HOUSEHOLD_1_541,WI_3,0,6.968750,6,1,2016,7.968750,1.0,4,-18.0,-13.0,-20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN


In [ ]:
train.drop(train.columns[0], axis=1, inplace=True)

In [ ]:
test.drop(test.columns[0], axis=1, inplace=True)

In [ ]:
#train.drop(['LentStart', 'LentWeek2', 'Purim End', 'Pesach End', 'ColumbusDay', 'MartinLutherKingDay'], axis=1, inplace=True)

In [ ]:
#test.drop(['LentStart', 'LentWeek2', 'Purim End', 'Pesach End', 'ColumbusDay', 'MartinLutherKingDay'], axis=1, inplace=True)

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [ ]:
y_train = train['Quantity_sold']
X_train = train.drop('Quantity_sold', axis=1)
y_test = test['Quantity_sold']
X_test = test.drop('Quantity_sold', axis=1)

In [ ]:
del train

In [ ]:
del test

Directly running the below cell on food df also crashes. So I am running it for X

In [ ]:
#X = X.loc[:, ~X.columns.str.contains('^Unnamed')]

In [ ]:
X_train[0:2444291]

,item_id,store_id,sell_price,wday,month,year,mrp,discount,week_number,PresidentsDay,LentStart,LentWeek2,StPatricksDay,Purim End,OrthodoxEaster,Pesach End,Cinco De Mayo,Mother's day,MemorialDay,NBAFinalsStart,NBAFinalsEnd,Father's day,IndependenceDay,Ramadan starts,Eid al-Fitr,LaborDay,ColumbusDay,Halloween,EidAlAdha,VeteransDay,Thanksgiving,Christmas,Chanukah End,NewYear,OrthodoxChristmas,MartinLutherKingDay,Easter
0,0,0,6.320312,1,1,2011,6.320312,0.000000,4,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
1,0,0,6.320312,2,1,2011,6.320312,0.000000,4,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
2,0,0,6.320312,3,1,2011,6.320312,0.000000,5,-21.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
3,0,0,6.320312,4,2,2011,6.320312,0.000000,5,-20.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
4,0,0,6.320312,5,2,2011,6.320312,0.000000,5,-19.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2444286,171,5,3.740234,6,11,2014,3.919922,0.179688,47,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,9.0,-7.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
2444287,171,5,3.740234,7,11,2014,3.919922,0.179688,47,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,10.0,-6.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
2444288,171,5,3.740234,1,11,2014,3.919922,0.179688,47,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,11.0,-5.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
2444289,171,5,3.740234,2,11,2014,3.919922,0.179688,47,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,12.0,-4.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0


In [ ]:
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
def label_encoding(X):
  d = defaultdict(LabelEncoder)
  categorical_mask = (X.dtypes == object)
  categorical_columns = X.columns[categorical_mask].tolist()
 
  print(X[categorical_columns].head())

  le = LabelEncoder()
  for cat in categorical_columns:
      le.fit(X[cat])
      #print(le.classes_)
      X[cat]=le.transform(X[cat])

  print(X[categorical_columns].head())
  return X

In [ ]:
X_train = label_encoding(X_train)
X_test = label_encoding(X_test)

           item_id store_id
0  HOUSEHOLD_1_001     CA_1
1  HOUSEHOLD_1_001     CA_1
2  HOUSEHOLD_1_001     CA_1
3  HOUSEHOLD_1_001     CA_1
4  HOUSEHOLD_1_001     CA_1
   item_id  store_id
0        0         0
1        0         0
2        0         0
3        0         0
4        0         0
           item_id store_id
0  HOUSEHOLD_1_001     CA_1
1  HOUSEHOLD_1_001     CA_1
2  HOUSEHOLD_1_001     CA_1
3  HOUSEHOLD_1_001     CA_1
4  HOUSEHOLD_1_001     CA_1
   item_id  store_id
0        0         0
1        0         0
2        0         0
3        0         0
4        0         0


As per what I read, one hot encoding would be needed for the store_id, and item_id variables, since the numbers assigned to them have no special significance and ordering between them would hurt performance.

For now I am skipping date for two reasons:
1. There are 1919 dates, one hot encoding would be too expensive
2. There are time series data, and for them order might help

In [ ]:
X_train.shape

(7340123, 37)

In [ ]:
X_train.fillna(1000, inplace=True)
X_test.fillna(1000, inplace=True)

Skip the next three cells also and try

In [ ]:
from sklearn.preprocessing import OneHotEncoder
 
ohe = OneHotEncoder(categories='auto', sparse=True)
 
# After applying one hot encoder, the output is no longer a data frame
X_encoded_train = ohe.fit_transform(X_train)
X_encoded_test = ohe.fit_transform(X_test)

In [ ]:
X_encoded.shape

NameError: ignored

Try again from here

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# fit model no training data
model_3 = xgb.XGBRegressor(n_estimators=600, max_depth = 10, verbosity =2)
model_3.fit(X_train[4895922:], y_train[4895922:], xgb_model=(input_dir+'hh1m2.model'))

[20:03:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:14:28] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1980 extra nodes, 0 pruned nodes, max_depth=10
[20:25:29] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1978 extra nodes, 0 pruned nodes, max_depth=10
[20:25:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1974 extra nodes, 0 pruned nodes, max_depth=10
[20:25:54] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1980 extra nodes, 0 pruned nodes, max_depth=10
[20:26:06] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1962 extra nodes, 0 pruned nodes, max_depth=10
[20:26:18] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1988 extra nodes, 0 pruned nodes, max_depth=10
[20:26:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roo

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=600,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=2)

1.    33.52
2.    30.75
3.    25.18

In [ ]:
model_2.save_model(input_dir + 'hh1m2.model')

In [ ]:
model_1.save_model(input_dir + 'hh1m1.model')

In [ ]:
y_pred = model_3.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 25.18%


In [ ]:
print(len(predictions))

457436


In [ ]:
zeros_real = y_test[y_test==0]
other_real = y_test[y_test!=0]
zeros_pred = []
other_pred = []
i=0
for y in y_test:
  if y==0:
    zeros_pred.append(predictions[i])
  else:
    other_pred.append(predictions[i])
  i=i+1

In [ ]:
print(len(other_real))
print(len(other_pred))

236280
236280


In [ ]:
mad_zeros = np.nanmean(np.abs(zeros_pred - zeros_real))
mape_other = np.nanmean(np.abs((other_pred-other_real)/other_real))

print(mad_zeros)
print(mape_other)
print(eval_results(predictions, y_test))

1.3525158
0.7566127
{'MAD': 1.5777464, 'MSE': 6.4921913, 'WMAPE': 1.0308166057742572, 'MD': -0.33383468, 'mean': 1.5305791411257532, 'SMAPE': 124.77971910626206}


In [ ]:
print(y_test)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold

In [ ]:
params = {'max_depth': [3, 6, 9], 'gamma': [0.5, 1, 1.5]}
grid = GridSearchCV(estimator=xgb.XGBRegressor(), param_grid=params, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=3)

In [ ]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed: 218.0min


In [ ]:
print('\n All results:')
print(grid.cv_results_)

In [ ]:
print('\n Best hyperparameters:')
print(grid.best_params_)

In [ ]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 6, 9]
        }

In [ ]:
param_comb = 50
random_search = RandomizedSearchCV(estimator=model, param_distributions=params, n_iter=param_comb, scoring='neg_mean_absolute_error' , n_jobs=-1, random_state=43, verbose=3)

In [ ]:
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed: 58.3min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed: 294.5min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 563.5min finished


[23:01:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=10, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=None,
                                          objective='reg:linear',
                                          random_state=0, reg_al...
                                          seed=None, silent=None, subsample=1,
                                          verbosity=1),
                   iid='deprecated', n_iter=50, n_job

In [ ]:
def eval_results(yhat_mean, y):
    temp_dict = {}
    temp_dict['MAD'] = np.nanmean(np.abs(y - yhat_mean))
    #temp_dict['MAPE'] = np.nanmean(np.abs((y-yhat_mean)/(y+1e-7)))
    temp_dict['MSE'] = np.nanmean(np.square(y - yhat_mean))
    temp_dict['WMAPE'] = np.nansum(np.abs(y - yhat_mean)) / np.nansum(y)
    temp_dict['MD'] = np.nanmean(y - yhat_mean)
    temp_dict['mean'] = np.nanmean(y)
    temp_dict['SMAPE'] = 100. * np.nanmean(np.abs(y - yhat_mean) / ((np.abs(y) + np.abs(yhat_mean)) / 2.))
    # print(temp_dict)
    return temp_dict

In [ ]:
print(eval_results(predictions, y_test))

In [ ]:
print(eval_results(predictions, y_test))

In [ ]:
def other_metrics(yhat, y):
  dic = {}
  false_positive_rate, true_positive_rate, thresholds = roc_curve(y, yhat)
  dic['false_pos'] = false_positive_rate
  dic['true_pos'] = true_positive_rate

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=100, max_depth=5)
regressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=5, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
# imputations and NaN 
# ordered variable 
# Random forest regression
